# (2)넘파이 배열의 그룹별 요약을 계산
- itertuples로 했던 작업을 넘파이 배열을 사용해 수행 가능
- 넘파이 배열을 사용해 데이터 부분집합에 대한 요약값을 얻을 수도 
---
- 표 데이터를 다룰 때 넘파이 배열은 유용하지만, 행 사이의 계산을 필요로 함
- 넘파이 배열 탐색이 판다스 데이터프레임 순회보다 빠름
- 일반적으로 데이터의 특정 세그먼트에 대해 재빨리 요약 값을 얻고자 할 때는 넘파이 배열을 선택하는 것이 합리적

In [10]:
import pandas as pd
import numpy as np
coviddaily = pd.read_csv('data/coviddaily720.csv', parse_dates=['casedate'])
ltbrazil = pd.read_csv('data/ltbrazil.csv')

In [11]:
loclist = coviddaily.location.unique().tolist()

### 넘파이 배열을 사용해 위치별 합계를 계산
- 위치와 신규 사례 데이터의 넘파이 배열 생성 : 위치별 전체 신규 사례 값 선택 > 합산

In [12]:
rowlist = []
casevalues = coviddaily[['location', 'new_cases']].to_numpy()

In [13]:
casevalues

array([['Afghanistan', 0.0],
       ['Afghanistan', 0.0],
       ['Afghanistan', 0.0],
       ...,
       ['Zimbabwe', 41.0],
       ['Zimbabwe', 16.0],
       ['Zimbabwe', 40.0]], dtype=object)

In [14]:
# 📍 반복문 사용하여 각 위치별 신규 사례 합산, 리스트에 저장 
for locitem in loclist:
    cases = [casevalues[j][1] for j in range(len(casevalues)) \
            if casevalues[j][0] == locitem]
    rowlist.append(sum(cases))

In [15]:
len(rowlist)

209

In [16]:
len(loclist)

209

In [17]:
rowlist[0:5]

[34451.0, 3371.0, 18712.0, 855.0, 483.0]

In [18]:
casetotals = pd.DataFrame(zip(loclist, rowlist), columns=['location', 'casetotals'])

In [19]:
casetotals.head()

,location,casetotals
0,Afghanistan,34451.0
1,Albania,3371.0
2,Algeria,18712.0
3,Andorra,855.0
4,Angola,483.0


### 지표온도 데이터 정렬, 온도에 누락값이 있는 행 드롭

In [21]:
ltbrazil = ltbrazil.sort_values(['station', 'month'])
ltbrazil.head()

,locationid,year,month,temperature,latitude,longitude,elevation,station,countryid,country,latabs
4,BR000352000,2019,1,NaN,-3.2,-52.2,112.0,ALTAMIRA,BR,Brazil,3.2
96,BR000352000,2019,2,NaN,-3.2,-52.2,112.0,ALTAMIRA,BR,Brazil,3.2
188,BR000352000,2019,3,NaN,-3.2,-52.2,112.0,ALTAMIRA,BR,Brazil,3.2
280,BR000352000,2019,4,NaN,-3.2,-52.2,112.0,ALTAMIRA,BR,Brazil,3.2
372,BR000352000,2019,5,NaN,-3.2,-52.2,112.0,ALTAMIRA,BR,Brazil,3.2


In [23]:
ltbrazil = ltbrazil.dropna(subset=['temperature'])

### 넘파이 배열을 사용해 연간 평균 온도 계산
- 한 기간과 다음 기간 사이에 큰 변화가 있는 행은 제외

In [25]:
prevstation = 'ZZZ'
prevtemp = 0
rowlist = []

tempvalues = ltbrazil[['station', 'temperature']].to_numpy()
tempvalues

array([['ALTAMIRA', 28.55],
       ['ALTAMIRA', 28.85],
       ['ALTAMIRA', 28.65],
       ...,
       ['VOTUPORANGA', 27.0],
       ['VOTUPORANGA', 25.15],
       ['VOTUPORANGA', 24.85]], dtype=object)

## 🎯 복습 필요
- 반복문 이해 & 결과값 다른거 확인(본문 246페이지)

In [30]:
for j in range(len(tempvalues)):
    station = tempvalues[j][0]
    temperature = tempvalues[j][1]
    
    if (prevstation!=station):    #이전 역과 현재 역이 다를 경우
        if (prevstation!='ZZZ'):  #이전 역이 'ZZZ'가 아닌 경우
            rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt,
                           'stationcnt':stationcnt})   # 이전 역의 정보를 딕셔너리 형태로 추가
        
        #새로운 역 정보 초기화
        tempcnt = 0
        stationcnt = 0
        prevstation = station # 현재역으로 업데이트
        
        if ((0 <= abs(temperature-prevtemp) <= 3) or (stationcnt == 0)):
            tempcnt += temperature
            stationcnt += 1
        
    prevtemp = temperature

In [31]:
rowlist.append({'station':prevstation, 'avgtemp':tempcnt/stationcnt, 'stationcnt':stationcnt})

In [32]:
rowlist[0:5]

[{'station': 'ALTAMIRA', 'avgtemp': 28.55, 'stationcnt': 1},
 {'station': 'ALTA_FLORESTA_AERO', 'avgtemp': 26.95, 'stationcnt': 1},
 {'station': 'ARAXA', 'avgtemp': 24.1, 'stationcnt': 1},
 {'station': 'BACABAL', 'avgtemp': 29.7, 'stationcnt': 1},
 {'station': 'BAGE', 'avgtemp': 25.1, 'stationcnt': 1}]

In [33]:
ltbrazilavgs = pd.DataFrame(rowlist)
ltbrazilavgs.head()

,station,avgtemp,stationcnt
0,ALTAMIRA,28.55,1
1,ALTA_FLORESTA_AERO,26.95,1
2,ARAXA,24.10,1
3,BACABAL,29.70,1
4,BAGE,25.10,1
